In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
from Hyprid import polFeature
import pandas as pd
import numpy as np
import sys
from rdkit.Chem import Descriptors

In [2]:
def hybConn(mol):
    sp3sp3 = 0
    sp3sp2 = 0
    sp3sp = 0
    sp2sp = 0
    sp2sp2 = 0
    spsp = 0
    sp3s = 0
    sp2s = 0
    sps = 0
    other = 0
    for bond in mol.GetBonds():
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        hyb1 = str(atom1.GetHybridization())
        hyb2 = str(atom2.GetHybridization())
        if   hyb1=="SP3" and hyb2=="SP3":sp3sp3+=1
        elif hyb1=="SP3" and hyb2=="SP2":sp3sp2+=1
        elif hyb1=="SP3" and hyb2=="SP" :sp3sp+=1
        elif hyb1=="SP3" and hyb2=="S"  :sp3s+=1
        elif hyb1=="S"   and hyb2=="SP3":sp3s+=1
        elif hyb1=="SP2" and hyb2=="SP3":sp3sp2+=1
        elif hyb1=="SP2" and hyb2=="SP2":sp2sp2+=1
        elif hyb1=="SP2" and hyb2=="SP" :sp2sp+=1
        elif hyb1=="SP2" and hyb2=="S"  :sp2s+=1
        elif hyb1=="S"   and hyb2=="SP2":sp2s+=1
        elif hyb1=="SP"  and hyb2=="SP3":sp3sp+=1
        elif hyb1=="SP"  and hyb2=="SP2":sp2sp+=1
        elif hyb1=="SP"  and hyb2=="SP" :spsp+=1
        elif hyb1=="SP"  and hyb2=="S"  :sps+=1
        elif hyb1=="S"   and hyb2=="SP" :sps+=1
        else:other+=1
    return [sp3sp3,sp3sp2,sp3sp,sp3s,sp2sp2,sp2sp,sp2s,spsp,sps,other]

In [3]:
def getMolDescriptors(mol):
    res = []
    for nm,fn in Descriptors._descList:
        if nm != 'Ipc':
            try:
                val = fn(mol)
                res.append(val)
            except:
                res.append(None)
                print(nm)
    return res

In [4]:
# Load and preprocess data
filename = "GTPase KRas.csv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename,sep=';')
hybd_names = ['FC','N','H','Cl','Br',
                 'I','P','F','Se','Si',
                 'S_sp3','S_sp2','S_sp',
                 'C_sp3','C_sp2','C_sp',
                 'N_sp3','N_sp2','N_sp',
                 'O_sp3','O_sp2','O_sp',
                 'I1','I2','I3']
bondhybd_names = ['sp3sp3','sp3sp2','sp3sp','sp3s',
                  'sp2sp2','sp2sp','sp2s','spsp','sps','other']
count = 0
dcr_names = [nm for nm,fn in Descriptors._descList]
dcr_names.remove('Ipc')
Cnames = ["ChEMBL ID","Smiles"]+hybd_names+bondhybd_names+dcr_names
print("Cnames", len(Cnames))
DF = pd.DataFrame(columns=Cnames)

Cnames 253


In [5]:
filename = "GTPase KRas.csv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename, sep=';')
df.dropna(subset=['Standard Value'], inplace=True)
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value,Value
0,CHEMBL5428861,NaN,NaN,703.65,2.0,6.64,25,C[C@H](Oc1nc2c(F)c(-c3cccc(Cl)c3Cl)c(CCC#N)cc2...,IC50,'<=',...,CHEMBL5365512,1,Scientific Literature,ACS Med Chem Lett,2023,NaN,TIME = 3.0 hr,INHIBITOR,NaN,100.000
1,CHEMBL5431853,NaN,NaN,622.72,2.0,5.77,3a,C[C@H](Oc1nc2c(F)c(-c3cccc4ccc(F)cc34)c(CCC#N)...,IC50,'<=',...,CHEMBL5365512,1,Scientific Literature,ACS Med Chem Lett,2023,NaN,NaN,INHIBITOR,NaN,100.000
2,CHEMBL5207895,NaN,NaN,476.49,0.0,3.71,Example 3,C=Cc1ncc2n1CCN(c1nc(=O)n(CC3CC3)c3nc(-c4c(O)cc...,IC50,'=',...,CHEMBL5131529,1,Scientific Literature,ACS Med Chem Lett,2022,NaN,TIME = 2.0 hr,INHIBITOR,NaN,100.000
3,CHEMBL3218637,NaN,NaN,1377.61,NaN,NaN,4,CC1(C)COC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H](CCCNC...,IC50,'=',...,CHEMBL3217533,1,Scientific Literature,Medchemcomm,2013,NaN,NaN,NaN,NaN,1.800
4,CHEMBL3218636,NaN,NaN,1499.80,NaN,NaN,2,CCCC[C@H]1NC(=O)[C@@H](Cc2ccc3ccccc3c2)NC(=O)[...,IC50,'=',...,CHEMBL3217533,1,Scientific Literature,Medchemcomm,2013,NaN,NaN,NaN,NaN,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,CHEMBL4476113,NaN,NaN,432.95,0.0,2.05,68; ARS-854,C=CC(=O)N1CC(N2CCN(C(=O)CNc3cc(C4(C)CC4)c(Cl)c...,IC50,'=',...,CHEMBL5149962,1,Scientific Literature,Eur J Med Chem,2021,NaN,NaN,INHIBITOR,NaN,1700.000
1070,CHEMBL5408905,NaN,NaN,817.33,2.0,7.62,40; Single diastereomer,C=C1CN2CCC[C@@]2(COc2nc(N3CCN(C(=O)/C=C/c4cccc...,IC50,'=',...,CHEMBL5329224,1,Scientific Literature,ACS Med Chem Lett,2024,NaN,TIME = 0.3333 hr,INHIBITOR,NaN,0.550
1071,CHEMBL5411023,NaN,NaN,436.94,0.0,4.17,19,C=CC(=O)N1CCN2Cc3ccc(-c4c(C)ccc5[nH]ncc45)c(Cl...,IC50,'=',...,CHEMBL5331133,1,Scientific Literature,J Med Chem,2023,NaN,TIME = 4.0 hr,INHIBITOR,NaN,8.290
1072,CHEMBL4456598,NaN,NaN,528.66,1.0,3.21,1,C=CC(=O)N1CCN(c2nc(OC[C@@H]3CCCN3C)nc3c2CCN(c2...,IC50,'=',...,CHEMBL4610106,1,Scientific Literature,J Med Chem,2020,CHEMBL3307543,NaN,NaN,NaN,142.000


In [6]:
# Iterate through ChEMBL IDs and SMILES
count = 0
for i, (chembl_id, smiles) in enumerate(zip(df["Molecule ChEMBL ID"], df["Smiles"])):
    try:
        count += 1
        if count == 100:
            count = 0
            print("Processing molecule number", i)

        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"Skipping {chembl_id}: Invalid SMILES '{smiles}'")
            continue

        mol = Chem.AddHs(mol)  # Add hydrogens
        status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())

        if status == 0:
            t = getMolDescriptors(mol)
            hyp = polFeature(mol)
            bhyp = hybConn(mol)
            I = [Descriptors3D.PMI1(mol), Descriptors3D.PMI2(mol), Descriptors3D.PMI3(mol)]

            DF.loc[i] = [chembl_id, smiles] + hyp + bhyp + I + t
            print(f"Processed {chembl_id} successfully")
        else:
            print(f"Skipping {chembl_id}: 3D conformer generation failed")

    except Exception as e:
        print(f"Error processing {chembl_id} ({smiles}): {repr(e)}")

# Save to CSV
output_filename = filename.split(".")[0] + "_Hyb_Features.csv"
DF.to_csv(output_filename, index=False)
print(f"Saved output to {output_filename}")

Processed CHEMBL5428861 successfully
Skipping CHEMBL5431853: 3D conformer generation failed
Processed CHEMBL5207895 successfully
Processed CHEMBL3218637 successfully
Processed CHEMBL3218636 successfully
Processed CHEMBL5083351 successfully
Processed CHEMBL5074453 successfully
Processed CHEMBL5081452 successfully
Processed CHEMBL5083090 successfully
Processed CHEMBL5076494 successfully
Processed CHEMBL2396992 successfully
Processed CHEMBL5171553 successfully
Processed CHEMBL5290394 successfully
Processed CHEMBL5273771 successfully
Processed CHEMBL4452819 successfully
Processed CHEMBL4847699 successfully
Processed CHEMBL4856294 successfully
Processed CHEMBL4867656 successfully
Processed CHEMBL4860021 successfully
Processed CHEMBL4872844 successfully
Processed CHEMBL4852554 successfully
Processed CHEMBL351706 successfully
Processed CHEMBL5407640 successfully
Processed CHEMBL4475526 successfully
Processed CHEMBL4857719 successfully
Processed CHEMBL5085560 successfully
Skipping CHEMBL407504

In [9]:
import pandas as pd

# Load the files
df_ic50 = pd.read_csv("GTPase KRas.csv", sep=";")  # Read TSV file
df_features = pd.read_csv("GTPase KRas_Hyb_Features.csv")

# Merge on the respective ID columns
merged_df = df_features.merge(df_ic50[['Molecule ChEMBL ID', 'Standard Value']],
                              left_on="ChEMBL ID",
                              right_on="Molecule ChEMBL ID",
                              how="left")

# Drop the duplicate ID column if needed
merged_df.drop(columns=["Molecule ChEMBL ID"], inplace=True)

# Save the merged file
merged_df.to_csv("merged_features_IC50_gtpase_kras.csv", index=False)

print("Merge completed. File saved as merged_features_IC50_gtpase_kras.csv")


Merge completed. File saved as merged_features_IC50_gtpase_kras.csv
